In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cudf
import glob
import gc
import os

In [4]:
import pickle

igfold = 1
sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356748


In [5]:
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})
df_type['type_'] = df_type['type_']+2

In [6]:
#os.system('rm -rf ./data_folds/fold_' + str(igfold) + '/data_gru/train/')
os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/data_gru/train/clicks/')
os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/data_gru/train/carts/')
os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/data_gru/train/orders/')
os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/data_gru/valid/')
os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/data_gru/test/')

0

In [7]:
from tqdm import tqdm

files_aid = sorted(glob.glob('../../data/' + '/train/interim/*.parquet')) + glob.glob('../../data/test.parquet') + glob.glob('./data/xgb_train_x.parquet')
for ifile in tqdm(range(len(files_aid))):
    out = []
    file = files_aid[ifile]
    df_tmp = cudf.read_parquet(file)
    if 'xgb_train_x.parquet' in file:
        print(file)
        df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
    else:
        df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
    out.append(df_tmp)
    df_aid = cudf.concat(out)
    del out
    gc.collect()
    df_aid['aid'] = df_aid['aid']+2
    df_aid = df_aid.merge(
        df_type,
        how='left',
        on='type'
    )
    df_aid.drop(['type'], axis=1, inplace=True)
    df_aid = df_aid.rename(columns={'type_': 'type'})
    out_clicks = []
    out_carts = []
    out_orders = []
    for ij in range(40):
        df_aid = df_aid.sort_values(['session', 'ts'], ascending=[True, False])
        df_aid['dummy'] = 1
        df_aid['rank'] = df_aid.groupby(['session']).dummy.cumsum()
        df_aid.drop(['dummy'], axis=1, inplace=True)
        df_valid_y = df_aid[df_aid['rank']==1]
        df_valid_x = df_aid[df_aid['rank']!=1]

        df_valid_x = df_valid_x.sort_values(['session', 'ts'])
        df_valid_agg = df_valid_x[
            df_valid_x['rank']<21
        ][['session', 'aid', 'type', 'rank']].groupby(['session']).agg(list)
        df_valid_agg = df_valid_agg.reset_index()
        df_valid_y = df_valid_y.rename(columns={
            'aid': 'target',
            'type': 'target_type'
        })
        df_valid_agg = df_valid_agg.merge(
            df_valid_y[['session', 'target', 'target_type']],
            how='left',
            on=['session']
        )
        df_valid_agg['session_len'] = df_valid_agg.aid.list.len()
        if df_valid_agg.shape[0]>0:
            if df_valid_agg[df_valid_agg['target_type']==2].shape[0]>0:
                out_clicks.append(df_valid_agg[df_valid_agg['target_type']==2])
            if df_valid_agg[df_valid_agg['target_type']==3].shape[0]>0:
                out_carts.append(df_valid_agg[df_valid_agg['target_type']==3])
            if df_valid_agg[df_valid_agg['target_type']==4].shape[0]>0:
                out_orders.append(df_valid_agg[df_valid_agg['target_type']==4])
        else:
            break
        df_aid = df_valid_x
    df_clicks = cudf.concat(out_clicks)
    df_clicks.to_parquet(
        './data_folds/fold_' + str(igfold) + '/data_gru/train/clicks/chunk_' + str(ifile) + '_' + str(0) + '.parquet'
    )
    df_carts = cudf.concat(out_carts)
    df_carts.to_parquet(
        './data_folds/fold_' + str(igfold) + '/data_gru/train/carts/chunk_' + str(ifile) + '_' + str(0) + '.parquet'
    )
    df_orders = cudf.concat(out_orders)
    df_orders.to_parquet(
        './data_folds/fold_' + str(igfold) + '/data_gru/train/orders/chunk_' + str(ifile) + '_' + str(0) + '.parquet'
    )

 99%|█████████▉| 130/131 [15:07<00:06,  6.86s/it]

./data/xgb_train_x.parquet


100%|██████████| 131/131 [15:13<00:00,  6.97s/it]


In [8]:
if True:
    files = glob.glob('../../data/test.parquet')
    out = []
    file = files[0]
    df_tmp = cudf.read_parquet(file)
    if 'xgb_train_x.parquet' in file:
        print(file)
        df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
    else:
        df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
    out.append(df_tmp)
    df_aid = cudf.concat(out)
    del out
    gc.collect()
    df_aid['aid'] = df_aid['aid']+2
    df_aid = df_aid.merge(
        df_type,
        how='left',
        on='type'
    )
    df_aid.drop(['type'], axis=1, inplace=True)
    df_aid = df_aid.rename(columns={'type_': 'type'})
    out_clicks = []
    out_carts = []
    out_orders = []
    for ij in range(1):
        df_aid = df_aid.sort_values(['session', 'ts'], ascending=[True, False])
        df_aid['dummy'] = 1
        df_aid['rank'] = df_aid.groupby(['session']).dummy.cumsum()
        df_aid.drop(['dummy'], axis=1, inplace=True)
        df_aid['rank'] = df_aid['rank']+1
        df_valid_x = df_aid
        df_valid_x = df_valid_x.sort_values(['session', 'ts'])
        df_valid_agg = df_valid_x[
            df_valid_x['rank']<21
        ][['session', 'aid', 'type', 'rank']].groupby(['session']).agg(list)
        df_valid_agg = df_valid_agg.reset_index()
        df_valid_agg['session_len'] = df_valid_agg.aid.list.len()

    df_valid_agg.to_parquet(
        './data_folds/fold_' + str(igfold) + '/data_gru/test/chunk_' + str(0) + '_' + str(0) + '.parquet'
    )

In [9]:
if True:
    files = glob.glob('./data/xgb_train_x.parquet')
    out = []
    file = files[0]
    df_tmp = cudf.read_parquet(file)
    if 'xgb_train_x.parquet' in file:
        print(file)
        df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
    else:
        df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
    out.append(df_tmp)
    df_aid = cudf.concat(out)
    del out
    gc.collect()
    df_aid['aid'] = df_aid['aid']+2
    df_aid = df_aid.merge(
        df_type,
        how='left',
        on='type'
    )
    df_aid.drop(['type'], axis=1, inplace=True)
    df_aid = df_aid.rename(columns={'type_': 'type'})
    out_clicks = []
    out_carts = []
    out_orders = []
    for ij in range(1):
        df_aid = df_aid.sort_values(['session', 'ts'], ascending=[True, False])
        df_aid['dummy'] = 1
        df_aid['rank'] = df_aid.groupby(['session']).dummy.cumsum()
        df_aid.drop(['dummy'], axis=1, inplace=True)
        df_aid['rank'] = df_aid['rank']+1
        df_valid_x = df_aid
        df_valid_x = df_valid_x.sort_values(['session', 'ts'])
        df_valid_agg = df_valid_x[
            df_valid_x['rank']<21
        ][['session', 'aid', 'type', 'rank']].groupby(['session']).agg(list)
        df_valid_agg = df_valid_agg.reset_index()
        df_valid_agg['session_len'] = df_valid_agg.aid.list.len()

    df_valid_agg.to_parquet(
        './data_folds/fold_' + str(igfold) + '/data_gru/valid/chunk_' + str(0) + '_' + str(0) + '.parquet'
    )

./data/xgb_train_x.parquet
